## Using SQL in Python

There are several ways to use SQL in Python. We can use SQLite, MS-SQL Server, Oracle, MySQL, etc.

In this notebook, I will just focus on [sqlite3](https://docs.python.org/3/library/sqlite3.html).

Steps to take:
- Create a connection using connect() method and pass the name of the database
- Use pd.read_sql() or pd.read_sql_query() to query from database
- Create a cursor object using cursor() method and execute SQL queries using execute() method
- Commit your changes using commit() method
- Close the connection using close() method

In [29]:
import sqlite3
import pandas as pd

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [30]:
con = sqlite3.connect('mental_health.sqlite') #any database with .db or .sqlite

In [37]:
# To break sql query to many lines, use docstring quote ('''xxx''')
pd.read_sql('''
SELECT *
FROM sqlite_master
WHERE type = 'table';
''',con)

,type,name,tbl_name,rootpage,sql
0,table,Answer,Answer,2,"CREATE TABLE ""Answer"" (\n\t""AnswerText"" VARCHA..."
1,table,Question,Question,1434,"CREATE TABLE ""Question"" (\n\t""questiontext"" VA..."
2,table,Survey,Survey,1438,"CREATE TABLE ""Survey"" (\n\t""SurveyID"" INTEGER ..."


In [34]:
# pd.read_sql_query() will work, too
# Double quote and single quote will work with one-line sql queries
survey = pd.read_sql_query("SELECT * FROM Survey",con)
survey

,SurveyID,Description
0,2014,mental health survey for 2014
1,2016,mental health survey for 2016
2,2017,mental health survey for 2017
3,2018,mental health survey for 2018
4,2019,mental health survey for 2019


In [35]:
ques = pd.read_sql('SELECT * FROM Question',con)
ques

,questiontext,questionid
0,What is your age?,1
1,What is your gender?,2
2,What country do you live in?,3
3,"If you live in the United States, which state ...",4
4,Are you self-employed?,5
...,...,...
100,Do you think that team members/co-workers woul...,114
101,"If yes, what condition(s) have you been diagno...",115
102,"If maybe, what condition(s) do you believe you...",116
103,Which of the following best describes your wor...,117


In [36]:
# Te be consistent, I will use pd.read_sql('''query''')
ans = pd.read_sql('''SELECT * FROM Answer''',con)
ans

,AnswerText,SurveyID,UserID,QuestionID
0,37,2014,1,1
1,44,2014,2,1
2,32,2014,3,1
3,31,2014,4,1
4,31,2014,5,1
...,...,...,...,...
236893,Other,2016,2689,117
236894,Support,2016,2690,117
236895,Back-end Developer,2016,2691,117
236896,DevOps/SysAdmin,2016,2692,117


In [7]:
pd.read_sql('''
SELECT SurveyID as 'SurveyYear'
,count(*) as 'ParticipantCount'
FROM Answer
GROUP BY SurveyID;
''',con)

,SurveyYear,ParticipantCount
0,2014,32760
1,2016,88238
2,2017,57456
3,2018,31692
4,2019,26752


In [8]:
# This kind of code in one-line can be confusing when thw query gets complicated
pd.read_sql_query("SELECT * FROM Question q INNER JOIN Answer a ON q.questionid == a.QuestionID", con)

,questiontext,questionid,AnswerText,SurveyID,UserID,QuestionID
0,What is your age?,1,-1,2014,391,1
1,What is your age?,1,-1,2014,716,1
2,What is your age?,1,-1,2014,1128,1
3,What is your age?,1,-1,2017,3447,1
4,What is your age?,1,-1,2017,3449,1
...,...,...,...,...,...,...
236893,Do you work remotely?,118,Sometimes,2016,2689,118
236894,Do you work remotely?,118,Sometimes,2016,2690,118
236895,Do you work remotely?,118,Sometimes,2016,2691,118
236896,Do you work remotely?,118,Sometimes,2016,2692,118


In [9]:
# This will be easier to read
df = pd.read_sql_query('''
SELECT a.QuestionID
, q.questiontext as QuestionText
, a.AnswerText
, SurveyID as SurveyYear
, UserID
FROM Question q
INNER JOIN Answer a
ON q.questionid == a.QuestionID;
''', con)
df

,QuestionID,QuestionText,AnswerText,SurveyYear,UserID
0,1,What is your age?,-1,2014,391
1,1,What is your age?,-1,2014,716
2,1,What is your age?,-1,2014,1128
3,1,What is your age?,-1,2017,3447
4,1,What is your age?,-1,2017,3449
...,...,...,...,...,...
236893,118,Do you work remotely?,Sometimes,2016,2689
236894,118,Do you work remotely?,Sometimes,2016,2690
236895,118,Do you work remotely?,Sometimes,2016,2691
236896,118,Do you work remotely?,Sometimes,2016,2692


In [10]:
type(df)

pandas.core.frame.DataFrame

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236898 entries, 0 to 236897
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   QuestionID    236898 non-null  int64 
 1   QuestionText  236898 non-null  object
 2   AnswerText    236898 non-null  object
 3   SurveyYear    236898 non-null  int64 
 4   UserID        236898 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 9.0+ MB


In [12]:
df.isna().sum()

QuestionID      0
QuestionText    0
AnswerText      0
SurveyYear      0
UserID          0
dtype: int64

In [13]:
#df[df['QuestionText'] == 'What is your age?'].groupby(['AnswerText'])['AnswerText'].count()
df[df['QuestionText'] == 'Do you work remotely?'].groupby(['AnswerText'])['AnswerText'].count()

AnswerText
Always       343
Never        333
Sometimes    757
Name: AnswerText, dtype: int64

In [16]:
# df is NOT a table of the database. To make it a table in the database, we need to create cursor, create table and commit it.
cursor = con.cursor()
con.execute('''
CREATE TABLE MentalHealth
AS
SELECT a.QuestionID
, q.questiontext as QuestionText
, a.AnswerText
, SurveyID as SurveyYear
, UserID
FROM Question q
INNER JOIN Answer a
ON q.questionid == a.QuestionID''')
con.commit() # the table is not saved to the database without commit()

In [17]:
pd.read_sql('''SELECT * FROM MentalHealth''',con)

,QuestionID,QuestionText,AnswerText,SurveyYear,UserID
0,1,What is your age?,-1,2014,391
1,1,What is your age?,-1,2014,716
2,1,What is your age?,-1,2014,1128
3,1,What is your age?,-1,2017,3447
4,1,What is your age?,-1,2017,3449
...,...,...,...,...,...
236893,118,Do you work remotely?,Sometimes,2016,2689
236894,118,Do you work remotely?,Sometimes,2016,2690
236895,118,Do you work remotely?,Sometimes,2016,2691
236896,118,Do you work remotely?,Sometimes,2016,2692


In [33]:
con.execute('''DROP TABLE MentalHealth''')
con.commit()

In [23]:
pd.read_sql('''
SELECT *
FROM sqlite_master
WHERE type = 'table';
''',con)

,type,name,tbl_name,rootpage,sql
0,table,Answer,Answer,2,"CREATE TABLE ""Answer"" (\n\t""AnswerText"" VARCHA..."
1,table,Question,Question,1434,"CREATE TABLE ""Question"" (\n\t""questiontext"" VA..."
2,table,Survey,Survey,1438,"CREATE TABLE ""Survey"" (\n\t""SurveyID"" INTEGER ..."
3,table,MentalHealth,MentalHealth,20765,"CREATE TABLE MentalHealth(\n QuestionID INT,\..."
4,table,Person,Person,14968,CREATE TABLE Person\n(PersonID integer PRIMARY...


In [21]:
#cursor = con.cursor()
cursor.execute('''
CREATE TABLE Person
(PersonID integer PRIMARY KEY
, Name text
, JobTittle Text
, Department text
, Salary real)
''')

In [22]:
cursor.execute('''
INSERT INTO Person
VALUES(1, 'John Doe', 'Manager', 'DataScientist', 70000 )
''')
con.commit()

In [24]:
person = pd.read_sql_query("SELECT * FROM Person",con)
person

,PersonID,Name,JobTittle,Department,Salary
0,1,John Doe,Manager,DataScientist,70000.0


In [25]:
cursor.close()
con.close()

That's it! That's how we intergrate SQL with Python. :D